## Question 2 (8 points)

Complete the function `estimate_phi` such that given a controlled $U$ gate where $U$ is an operator with eigenvector $|1\rangle$ and eigenvalue $e^{2\pi i \phi}$, it estimates and returns $\phi$.

The function `estimate_phi` has

- Input: Controlled $U$ gate
- Returns: Estimate for phi 

You are given iqft and qpe algorithms and you can use them in your solution. Let the size of the first register equal 10.

In [2]:
def iqft(n,qubits,circuit):
    
    #Swap the qubits
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW)
     
    #For each qubit
    for i in range(n-1,-1,-1):
        #Apply CR_k gates where j is the control and i is the target
        k=n-i #We start with k=n-i
        for j in range(n-1,i,-1):
            #Define and apply CR_k gate
            crk = CZPowGate(exponent = -2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]),strategy = InsertStrategy.NEW)
            k=k-1 #Decrement at each step
            
        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]),strategy = InsertStrategy.NEW)

def qpe(t,control, target, circuit, CU):
    
    #Apply Hadamard to control qubits
    circuit.append(cirq.H.on_each(control))
    
    #Apply CU gates
    for i in range(t):
        #Obtain the power of CU gate 
        CUi = CU**(2**i)
        #Apply CUi gate where t-i-1 is the control
        circuit.append(CUi(control[t-i-1],*target))
        
    #Apply inverse QFT
    iqft(t,control,circuit)

In [3]:
import cirq
from cirq.circuits import InsertStrategy
from cirq import H, SWAP, CZPowGate, X
def estimate_phi(mystery):
    

    #Create cirucit
    circuit = cirq.Circuit()
    # YOUR CODE HERE
    t=10
    control = cirq.LineQubit.range(t)
    target = [cirq.LineQubit(t)]

    circuit.append(X(target[0])) # Initialize target register to |1>

    qpe(t,control,target,circuit,mystery) # Apply QPE

    circuit.append(cirq.measure(*control,key='result')) # Measure control register


    # Initialize target register to |1>
    circuit.append(cirq.X(target[0]))

    print("Running QPE to estimate φ...")
    

    s=cirq.Simulator()
    samples=s.run(circuit, repetitions=1000)
    
    #Most frequent observation
    freq = list(samples.histogram(key='result').keys())[0]

    return freq/(2**t)

In [4]:
import math
# You can use this code to test your function by different operators
def test_qpe(phi):
    operator = CZPowGate(exponent=2*phi)
    return estimate_phi(operator)
   
assert(math.isclose(test_qpe(0.23),0.23,rel_tol=1e-2))


Running QPE to estimate φ...
